# Lec4 統計の基礎
<div dir='rtl'>
2022.4岩政
</div>

## 講義の内容
- 統計とは
  - 母集団と標本
  - 記述統計量
  - 正規分布、様々な確率分布
- 推定
  - 点推定、区間推定と信頼区間
- 仮説検定
  - 帰無仮説、対立仮設、有意水準、相関


統計分布について一通り確率密度関数から検定までが実装されている．scipy.statsを使います。

参考：
https://github.com/rasbt/data-science-tutorial

## 要約統計量 (Summary Statistics) 

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

Irisデータセットをロードし、'sepal_length'(がく片の長さ cm)の分布をヒストグラムで確かめます。

In [ ]:
# read dataset　
#df = pd.read_csv('./datasets/iris/iris.csv')
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2022/main/notebooks/datasets/iris/iris.csv')

def histo():
    # create histogram
    bin_edges = np.arange(0, df['sepal_length'].max() + 1, 0.5)
    fig = plt.hist(df['sepal_length'], bins=bin_edges)

    # add plot labels
    plt.xlabel('count')
    plt.ylabel('sepal length')
    
    
histo()

'sepal_length'の方がfloat64であることを確かめます。(N.A等が入ってないかどうか)

In [ ]:
x = df['sepal_length'].values
x.dtype

### 平均
平均を計算します

In [ ]:
np.sum(i for i in x) / len(x)

numpyのmean/0関数でも計算できますが、値は同じではありませんん。

In [ ]:
x_mean = np.mean(x)
x_mean

ヒストグラムに、平均の線（縦線）を重ね書きします。

In [ ]:
histo()
plt.axvline(x_mean, color='darkorange')

### 分散
分散を計算します。最初は、公式どおりに計算します


In [ ]:
np.sum([(i - x_mean)**2 for i in x]) / (len(x) - 1)

numpyのvar/2を使っても計算できます。ddofは

Bessel's correction(nの代わりにn-1で割ること)を考慮している

ddof:初期値0　分散を計算する際、平均との偏差の2乗の和をN-ddofで割ります。初期値ではddof=0なのでデータ数であるNで割ることになる

In [ ]:
var = np.var(x, ddof=1)
var

In [ ]:
df['sepal_length'].var() # note that Bessel's correction is the default

それでは平均±分散を縦線で重ねます

In [ ]:
histo()
plt.axvline(x_mean + var, color='darkorange')
plt.axvline(x_mean - var, color='darkorange')

### 標準偏差：

　numpyのstd関数でも計算できます。std関数はデフォルトでBessel's correction が入ってます

In [ ]:
(np.sum([(i - x_mean)**2 for i in x]) / (len(x) - 1))**0.5

In [ ]:
np.sqrt(np.var(x, ddof=1))

In [ ]:
np.sqrt(np.var(x, ddof=0))

In [ ]:
std=df['sepal_length'].std() # note that Bessel's correction is the default
std

In [ ]:
histo()
plt.axvline(x_mean + std, color='darkorange')
plt.axvline(x_mean - std, color='darkorange')

Min/Max

In [ ]:
np.min(x),np.max(x)

### モード（最頻値):

いったんlistに変換すると list.count(値)で、その値をもつデータ数を数え上げられます

関数 max(set(list_name), key = list_name.count) は、指定されたリストで最大回数発生する要素を返します。

lst.coun(x)で lst中のxの数をカウント、lstのそれぞれのuniqueな要素aに対して、lst.count(a)をしてその結果が最大となるaを返す。


In [ ]:
lst = list(x)
mode = max(set(lst), key=lst.count)
mode

In [ ]:
set(lst)

最頻値に何個データがあったか、

In [ ]:
lst.count(mode)

stasのmode関数でも求められます

In [ ]:
stats.mode(x)

### パーセント点

25パーセント点と75パーセント点:

- まずはxをソートしてyを得ます
- y.shape[0]は、yの要素数を返します。要素数の25%/75&のところの値を得ます
  

In [ ]:
y = np.sort(x)
percentile_25th = y[round(0.25 * y.shape[0]) + 1]
percentile_25th

In [ ]:
percentile_75th = y[round(0.75 * y.shape[0]) - 1]
percentile_75th

numpyのpercentile関数を使ってもば求まります。

```
補間モードを用いて異なるパーセンタイルを計算することができます。補間モードには linear、lowower、higher、midpoint、nearest があります。これらの補間モードは、パーセンタイルが 2つのデータ点 i と j の間にある場合に使用されます。
```

In [ ]:
np.percentile(x, q=[25, 75], interpolation='lower')

pandasのDataFrameのquntile関数を使っても求められます

In [ ]:
df['sepal_length'].quantile(0.25, interpolation='lower')

In [ ]:
df['sepal_length'].quantile(0.75, interpolation='lower')

25パーセント点と75パーセント点をプロットしてみます

In [ ]:
histo()
plt.axvline(percentile_75th, color='darkorange')
plt.axvline(percentile_25th - var, color='darkorange')

中央値 (50パーセント点でもある):

In [ ]:
x = np.sort(x)

tmp = round(0.5 * x.shape[0])

if x.shape[0] % 2:
    median = x[tmp - 1]
else:
    median = x[tmp - 1] + (x[tmp] - x[tmp - 1]) / 2.
    
median

numpyにはmedian関数がある

In [ ]:
np.median(x)

In [ ]:
histo()
plt.axvline(median, color='darkorange')

# 確率分布

## 正規分布を知る

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.stats
from scipy.stats import norm  # normal distribution, 正規分布

import matplotlib.pyplot as plt
%matplotlib inline

### 正規分布のグラフ pdf（probability density function）

In [ ]:
m = 5   # 平均値
std = 2 # 標準偏差
std2 = 4
x = np.arange( -5, 15, 0.01)
y = norm.pdf(x, loc=m, scale=std)
y2 = norm.pdf(x, loc=m, scale=std2)
#y = (1 / np.sqrt(2 * np.pi * std*std ) ) * np.exp(-(x-m) ** 2 / (2 * std*std) ) #正規分布の式

fig = plt.subplots(figsize=(8,3))
plt.plot(x,y,label='std=2')
plt.plot(x,y2,label='std=4')
plt.xlabel('x')
plt.ylabel('probabiity density')
plt.legend()

### 累積確率(Cumulative Distribution Function)


In [ ]:
x = np.linspace(-4, 4, 100)
plt.plot(x, stats.norm.pdf(x))
#plt.plot(x, stats.t.pdf(x,99))
plt.xlabel('x')
plt.ylabel('P')
plt.title('Normal Distribution')

In [ ]:
#y_cdf = stats.t.cdf(x,100-1)
y_cdf = stats.norm.cdf(x)
plt.plot(x, y_cdf)
plt.xlabel('x')
plt.ylabel('P')
plt.title('Cumulative Distribution Function')

### パーセント点と確率の計算
パーセント点（pp:percent point)$z_{\alpha}$ を求めるには，$1-\alpha$をppf ( percent point function )に与える。<br>
確率を求めるには，パーセント点$z_{\alpha}$をcdf (cumulative density function)　に与える。<br>
なお，標準正規分布（平均値 $m = 0$,  標準偏差 $\sigma=1$ ）を対象とする<br>

#### 片側の場合
標準正規分布$N(0,1)$のグラフで，$z_{\alpha} \le x < \infty$ の確率 $\alpha$（面積）をユーザが与える。このとき，  
$- \infty < x \le z_{\alpha} $の確率（面積）は $1-\alpha$となる。
この値をscipy.stats.norm.ppf に与えれば，$z_{\alpha}$が求まる。

In [ ]:
m = 0
std = 1
alpha = 0.05
prob = 1 - alpha
z_alp = norm.ppf(prob, loc=m, scale=std)
print('パーセント点＝', z_alp) #

In [ ]:
# グラフ書いてみるよ
m = 0
std = 1
alpha = 0.05

x = np.arange( -3, 3, 0.01)
y = norm.pdf(x, loc=m, scale=std)

fig = plt.subplots(figsize=(5,3))
plt.plot(x,y,label='std=1')

prob = 1 - alpha
z_alp = norm.ppf(prob, loc=m, scale=std)
plt.axvline(z_alp, color='orange',label="z_alp={0}".format(z_alp))
plt.title("Percent point z_alp for alpha=0.05")
plt.legend()

isf (Inverse survival function，生存関数の逆関数）を用いると，1-alphaの計算が不要で，上記と同じパーセント点を得る

In [ ]:
z_alp2 = norm.isf(alpha, loc=m, scale=std)
print('isfを用いたパーセント点', z_alp2)

In [ ]:
m = 0
std = 1.0
z_alp = 1.96
prob = norm.cdf(z_alp, loc=m, scale=std)
print('確率＝',prob)

#### 両側の場合
区間[a b]の確率を求める。
1. $- \infty < x \le z_{b} $の確率$P_a$を求める  
2. $- \infty < x \le z_{a} $の確率$P_b$を求める
3. $P_a - P_b$を計算

In [ ]:
za = 1.96
zb = -1.96
pa = norm.cdf(za, loc=0, scale=1) # loc is mean
pb = norm.cdf(zb, loc=0, scale=1) # scale is standard deviation
p = pa - pb
print('p=',p)

In [ ]:
# グラフ書いてみるよ
za = 1.96
zb = -1.96
x = np.arange( -3, 3, 0.01)
y = norm.pdf(x, loc=0, scale=1)

fig = plt.subplots(figsize=(5,3))
plt.plot(x,y,label='std=1')

pa = norm.cdf(za, loc=0, scale=1) # loc is mean
pb = norm.cdf(zb, loc=0, scale=1) # scale is standard deviation
p = pa - pb

plt.axvline(za, color='orange',label="za={0}".format(za))
plt.axvline(zb, color='yellow',label="zb={0}".format(zb))
plt.title("Probability for p=pa-pb={0}".format(p))
plt.legend()

#### 検定では，$\alpha$が初めに与えられる。
これに基づき，片側（z_alp），両側（za, zb）を求めることが多い。  
片側の場合は既に述べた。  
両側の場合，正規分布が対称で，かつ，（za, zb）が原点を中心とした左右対称という前提があり，この場合，intervalを用いることができる。<br>
下記の例では，両側にz_alp/2=0.025 があり，この二つを足して0.05となることに留意。

In [ ]:
za,zb = norm.interval(alpha=0.95, loc=0, scale=1)
print('za=',za,'  zb=',zb)

横軸が $x$,  縦軸が頻度（データ数Nが多いほど，縦軸のスケールも大きくなる）<br>
・平均値を中心とした分布となる<br>
・データ数100程度では，正規分布の形とは言えず，データ数を非常に多くして，ようやく理想形に近くなる<br>
・標本平均値，標本標準偏差もデータ数がかなり多くないと，真値に近づかない<br>
・標準偏差の計算 x.std(ddof=1) のddof=1は“不偏標準偏差”を求めるとき，すなわち，1/(N-1)という除算を行う<br>
・このddofを指定しないと， 1/N を用いた計算を行い，不偏とならない<br>
・標準偏差はばらつきの指標となる

In [ ]:
np.random.seed(123) # scipyと共通，乱数発生の再現性を得る

mean = 2.0 # mean, 平均値
std = 3.0  # standar deviation, 標準偏差
#for N in [100, 10000]:
for N in [100]:
    x = scipy.stats.norm.rvs(loc=mean, scale=std, size=N) # rvs:Random variates
    print('N = %d  mean = %f  std = %e' % (N, x.mean(), x.std(ddof=1))) 
    plt.figure()
    plt.hist(x, bins=20)
    plt.title('$N = %i$' % (N) )


### 中心極限定理
一様乱数を用いる，この区間[a,b]の平均値は$\mu = \frac{a+b}{2}$, 分散$\sigma^2 = \frac{(b-a)^2}{12}$である。
このn個（幾つかの値）の標本平均をN個（これは一定とする）発生させ，その分布を見ると，正規分布$N(\mu, \sigma^2/n )$に近づく。ここでは，標準正規分布に正規化する。
他の分布でも試してみられたい。

In [ ]:
N = 2000
y= np.zeros(N)
for n in [5, 5000]:
    for i in range(N):
        x = scipy.stats.uniform.rvs(size=n)
        y[i] = (x.mean() - 1/2)/(np.sqrt(1/12)/np.sqrt(n))
    plt.hist(y, bins=20, range=(-4,4), density=True)

xx = np.arange(-4, 4, 0.01)
nrm = scipy.stats.norm.pdf(xx, loc=0.0, scale=1.0 )
plt.plot(xx, nrm, c='k')


## ポアソン分布
$$
P(X=k) = \exp (-\lambda t) \frac{(\lambda t)^k}{k !}
$$
scipy.stats.poisson  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.stats import poisson
from scipy.stats import uniform

import matplotlib.pyplot as plt
%matplotlib inline


### 確率質量関数（pmf: Probability mass function）のグラフ
離散確率変数の場合の呼称 ，連続確率変数の場合は確率密度関数 (pdf: Probability density function)

In [ ]:
fig = plt.subplots(figsize=(8,4))
k = np.arange(0,16)

for lamb in range(1,6):
    p = poisson.pmf(k, lamb)
    plt.plot(k, p, label='lamb='+str(lamb))

plt.xlabel('k')
plt.ylabel('Probability mass function')
plt.legend()


#### 例：交通事故問題
交通事故　平均2.4件／日のとき，交通事故が2件／日　以下となる確率を求める。

In [ ]:
lamb = 2.4
psum = 0
for k in [0, 1, 2]:
    p = poisson.pmf(k, mu=lamb)
    psum = psum + p
print('sum of p =',psum)

#### ポアソン到着モデル
$$ 
  t_{arrive} = -\frac{1}{\lambda} \log_{e} P_{arrive} (T) 
$$

Num人分の到着時刻を得る

ポアソン分布に従う確率変数の生成に、逆関数を用いた乱数生成を用いた

[橋本]「3.4.4 逆関数を用いた乱数生成」

In [ ]:
np.random.seed(123)

Num=30 # 　Num人分の到着時刻を得る
t_arrive = np.zeros(Num)
lamb = 1

sum = 0.0
for i in range(Num):
    sum = sum - (1/lamb) * np.log( uniform.rvs(size=1) )
    t_arrive[i] = sum

fig, ax = plt.subplots(figsize=(6,3))
ax.vlines(t_arrive, ymin=0, ymax=1)
ax.set_xlabel('time[k]')
plt.tight_layout()

In [ ]:
t_arrive

確率の種設定(np.random.seed(123))しなければ、毎回、到着の様子は変わる

In [ ]:
Num=30 # the number of arraivl, Num人分の到着時刻を得る
t_arrive = np.zeros(Num)
lamb = 1

sum = 0.0
for i in range(Num):
    sum = sum - (1/lamb) * np.log( uniform.rvs(size=1) )
    t_arrive[i] = sum

fig, ax = plt.subplots(figsize=(6,3))
ax.vlines(t_arrive, ymin=0, ymax=1)
ax.set_xlabel('time[k]')
plt.tight_layout() 


## 平均値の検定　（母分散が未知） t検定
$$
t = \frac{\hat{\mu} - \mu_0}{\sqrt{\sigma^2 \big/ N}}
$$
上記の$t$検定量は自由度$ df = N - 1 $の$t$分布に従う（ $df$: Degree of Freedom）<br>
scipy.stats.t https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html <br>

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline


#### 例4.3：あるクラス，テストの平均点と補講の効用（片側検定）


あるクラスで数学の平均を向上させるための補講の前後でテストを行い効用を知りたい。前後の得点差は1,-1,-2,3,-1,5,4,0,7,-1であった。有意水準$\alpha=5$%

In [ ]:
data = np.array([1, -1, -2, 3, -1, 5, 4, 0, 7, -1])
m = np.average(data) # mean
s = np.std(data, ddof=1) # std, ddof=1 : unbiased
N = len(data) # the number of sample
print(m,s,N)

In [ ]:
alp = 0.05
# 自由度N-1のt分布で、alphaに対するパーセント点を求める(1-alpa)をppf(percent point function)に入れて求める->talp
talp = stats.t.ppf((1-alp),N-1)
print('talp (alpha=0.05, df=%d) =%f' %((N-1),talp))

m0 = 0 # null hypothesis
# 母関数の平均が0が帰無仮説
# t値は、は talpより大？小？
t = (m-m0)/(s/np.sqrt(N))
print('t=', t)

talp > t より，H0は棄却できない。
通常は，talp, t のようなpp(percent point)を求めるよりは，
p値(p value)を求める。これを次に示す

In [ ]:
prob = stats.t.cdf(t,N-1)
print('p value=',1-prob)

有意水準を0.05 とおくと，これよりp valueの方が大きいので，H0を棄却できない。

- 補講の効果がなかった＝帰無仮説$H_0(\mu_0=0)$、対立仮設は$\mu_0\gt0$(片側検定)
- p値(0.078)は設定した$\alpha$(0.05)より大きい→　$H_0$を棄却できない
- 分母の$\hat{\sigma}$（ばらつき）が大きいとtの値は小さい
- ばらつきを小さくすれば棄却できる、補講の効果を立証できる

## 分散の検定　（母平均が未知） $\chi ^2$検定
$$
\chi^2 = \frac{N-1}{\sigma_0^2} \hat{\sigma}^2 \sim \chi^2 (N-1)
$$
上記の$\chi^2$検定量は自由度$ df = N - 1 $の$\chi^2$分布に従う（ $df$: Degree of Freedom）<br>
scipy.stats.chi2 https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html <br>
$\chi^2$分布は非対称の分布形状である。

#### 例4.4：精密部品の直径（両側検定）

アルファ社の精密部品について、規格では分散は　$1\times 10^{-7}$　mm以下としている。このとき母標準偏差は、この規格からずれているか有意水準5%で検定する。$H_0$を$\mu_0=1.54$,$H_1$を$\mu_0\ne1.54$とする

In [ ]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
m = np.mean(data2)        # mean
s = np.std(data2, ddof=1) # std, ddof=1 : unbiased
N = len(data2)            # 
df = N - 1               # DoF (degree of freedom)
m0 = 1.54                 # H0 (null hypothesis)
print('sample mean =',m,'  std = ',s,' The number of data = ',N)

In [ ]:
t = (m-m0)/(s/np.sqrt(N))   # サンプルから求まるpp値
prob = stats.t.cdf(t, df)
if t >=  0:
    p = 1 - prob
else:
    p = prob

print('t = ',t)
print('p value =',2*p)

In [ ]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
m = np.mean(data2)        # mean
s = np.std(data2, ddof=1) # std, ddof=1 : unbiased
m0=1.54
N = len(data2) 
t = (m-m0)/(s/np.sqrt(N))   # サンプルから求まるpp値
prob = stats.t.cdf(t, N-1)
p= (1-prob) if t >= 0 else prob
pvalue=2*p
pvalue

In [ ]:
prob,t,m,s

注意：上記のように，tが負の値をとるときは，p = prob とする。  
$\alpha=0.05$と置くならば，p value $< \alpha$ よりH0を棄却

p値(0.0449)は0.05より小さいので、$H_0$を棄却して$H_1$を採用、つまり規格通りでない

実は，次の関数を用いると計算は楽<br>
t, p = scipy.stats.ttest_1samp(data, m0)<br>
data: 1群のサンプルデータ<br>
m0: 帰無仮説H0で仮定した平均値<br>
t: t値<br>
p: p値，両側検定を前提としている。片側検定ではこの半分の値を用いる<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html

In [ ]:
t, p = stats.ttest_1samp(data2, m0)
print('t = ',t)
print('both side p = ',p)
print('one  side p = ',p/2)

## 2標本の平均値の差の検定

2標本，両方の母分散が共に未知である場合を扱う。この場合ウェルチのt検定（Welch's t-test）を用いる。  
統計検定量$t$は複雑ゆえ，次を参照されたい：https://en.wikipedia.org/wiki/Welch%27s_t-test  
この自由度は，ウェルチ-サタスウェイトの式（Welch–Satterthwaite equation）より近似的に求められる。
https://en.wikipedia.org/wiki/Welch%E2%80%93Satterthwaite_equation  

自由度も自動的に計算する関数が次のscipy.stats.ttest_ind,  入力で equal_var = False を指定する  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

#### 例4.6 ２つの体温計の性能検定
２つの体温計s1とs2の測定精度にさがあるかを確かめた測定結果から、両方の平均値の差に有意差があるか否かの検定を行う。帰無仮説=$\hat{\mu_x}=\hat{\mu_y}$、対立仮設は$\hat{\mu_x}\ne\hat{\mu_y}$(両側検定)とする。

In [ ]:
s1 = np.array([37.1, 36.7, 36.6, 37.4, 36.8, 36.7, 36.9, 37.4, 36.6, 36.7])
s2 = np.array([36.8, 36.6, 36.5, 37.0, 36.7, 36.5, 36.6, 37.1, 36.4, 36.7])

stats.ttest_indを使い、対応がないt検定を行う。

２つの集団からの各対象から、１つずつ値を抜き出してきて、平均値の差が有意かどうかを調べる検定。

In [ ]:
t, p = stats.ttest_ind(s1, s2, equal_var = False)
print('t = ',t, ' p value = ',p)

p値をみると$\alpha=5$とすると、これより大きいので帰無仮説$H_0$を棄却できない、２つの体温計の平均値が等しいという仮説は棄却できない。

## 相関関係、無相関の検定

In [ ]:
from scipy import stats
import numpy as np
x=np.array([168,172,181,179,166,185,177,176,169,161])
y=np.array([111,125,129,120,126,133,130,116,118,115])
stats.pearsonr(x,y)

In [ ]:
plt.scatter(x,y)